In [3]:
#libraries
import os
import glob
import numpy as np
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam,lr_scheduler
from torch.autograd import Variable
import torchvision
import pathlib

In [4]:
#checking for device
device=("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
print(device)

cpu


In [6]:
#transforms
transform=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), #0-255 to 0-1, numpy to tensors
    transforms.Normalize ([0.5,0.5,0.5],
                          [0.5,0.5,0.5] )
]) 


In [7]:
#hyperparameter
epochs=1
batch_size=64
learning_rate=0.001
#dataloader
train_path="D:\DataScienceTutorial\PytorchTutorial\data\seg_train\seg_train"
test_path="D:\DataScienceTutorial\PytorchTutorial\data\seg_test\seg_test"
train_data=DataLoader(torchvision.datasets.ImageFolder(train_path,transform=transform),batch_size=batch_size,shuffle=True)
test_data=DataLoader(torchvision.datasets.ImageFolder(test_path,transform=transform),batch_size=batch_size,shuffle=False)

In [8]:
#classes
root=pathlib.Path(train_path)
classes=sorted([j.name.split("/")[-1] for j in root.iterdir()])

In [9]:
print(len(classes))

6


In [33]:
#CNN
class ImageClassification(nn.Module):
    def __init__(self,num_classes):
        super(ImageClassification,self).__init__()
        #output size after convolution filter is ((wide-filter+2padding)/s)+1
        #input shape (256,3,150,150) (batch_size, channel,wide,height)
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1, padding=1)
        #new shape (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        # shape=(256,12,75,75)
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #shape=(256,20,75,75)
        self.relu2=nn.ReLU()
        #shape=(256,20,75,75)
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
         #new shape (256,32,75,75)
        self.bn2=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        # shape=(256,32,75,75)
        
        self.fc=nn.Linear(in_features=75*75*32,out_features=num_classes)
        
        # feed forward function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool(output)

        output=self.conv2(output)
        output=self.relu2(output)

        output=self.conv3(output)
        output=self.bn2(output)
        output=self.relu3(output)
        # output shape will be (256,32,75,75)
        output=output.view(-1,75*75*32)
        output=self.fc(output)
        return output

In [34]:
model=ImageClassification(len(classes)).to(device)

In [35]:
#optimizer and loss
optimizer=Adam(model.parameters(),lr=learning_rate,weight_decay=0.0001)
#scheduler=lr_scheduler(optimizer,step_size=5,gamma=0.01)
criterion=nn.CrossEntropyLoss()

In [36]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+"/**/*.jpg"))
test_count=len(glob.glob(test_path+"/**/*.jpg"))

In [37]:
print(train_count,test_count)

14034 3000


In [40]:
#model training and saving best model
best_accuracy=0.0
for epoch in range (epochs):
    #evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    for i, (images,labels) in enumerate(train_data):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
        optimizer.zero_grad()
        outputs=model(images)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        train_loss +=loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        train_accuracy +=int(torch.sum(prediction==labels.data))
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    #Evaluation on testing dataset
    model.eval()
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_data):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'IntelImageFolder.model')
        best_accuracy=test_accuracy
    

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\DataScienceTutorial\\PytorchTutorial\\data\\seg_train\\seg_train\\buildings\\18814.jpg'